## Introduction
The machine learning problem we are trying to solve is predicting the College Football Playoff Commitee's top 25 teams ranking. This matters beacuse th CFP Commitee's rankings are riddled with controversey year after year, as teams are constantly frustrated with their ranking. These rankings matter heavily, as they determine how prestigious of a bowl game teams play in, as well as who gets to compete for a national champsionship. There is very little information on what the commitee considers when they rank the teams, so our model could be used to give teams insight into what particular statistics the committe might value the most when they determine which teams should be ranked higher than others. Teams could then place emphasis on say making sure they have good passing offense or a positive turnover margin if the model shows teams who perform well in those categories are ranked well by the commitee.

The dataset we are using we got from Kaggle at the following link: https://shorturl.at/glT68

It holds data on over a 140 statistical categories(our features) on all FBS teams(the best 130 or so teams in the country) for every year the College Playoff Committe has existed, which is from 2014 to the present. Examples of features in our dataset are total points scored, offensive yards per play, sacks, and many more. We have over 800 records in our dataset.



## Data Cleaning

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import sklearn

In [ ]:
first_year = 14 # the first dataset is from 2014
num_years = 7 # the datasets go from 2014-2020
df_dict = {} # holds individual dataframes for each year

Load every dataset into a dataframe and maps the year to its corresponding dataframe.

In [ ]:
for year in range(first_year, first_year + num_years):
    df = pd.read_csv("cfb" + str(year) + ".csv")
    df_dict[year] = df

Adds the year the datasets are from as a column in each dataframe. We will drop the year feature eventually, but it will be helpful when we feature engineer later after merging the dataframes into one big dataframe.

In [ ]:
for year in range(first_year, first_year + num_years):
    df = df_dict[year]
    df['year'] = year

There were some issues with the kick return data being in wrong columns in some datasets, so we swapped around the data and fixed some column name issues as well.

In [ ]:
for year in range(first_year + 2, first_year + num_years):
    df = df_dict[year]
    temp_data = df['Avg.Yard.per.Kickoff.Return'].copy()
    df['Avg.Yard.per.Kickoff.Return'] = df['...40']
    df['...40'] = temp_data
    temp_data = df['Kickoff.Return.Touchdowns'].copy()
    df['Kickoff.Return.Touchdowns'] = df['...40']
    df['...40'] = temp_data
    temp_data = df['Kickoff.Return.Yards'].copy()
    df['Kickoff.Return.Yards'] = df['...40']
    df['extra column'] = temp_data
    df = df.drop(columns='...40')

Merges each individual dataframe into one big dataframe.

In [ ]:
key_set = sorted(list(df_dict.keys()))
while len(key_set) > 1:
  df0 = df_dict[key_set[0]]
  df1 = df_dict[key_set[1]]
  shared_columns = df0.columns.intersection(df1.columns)
  df0 = df0.loc[:, shared_columns]
  df1 = df1.loc[:, shared_columns]
  df0 = pd.concat([df0, df1], ignore_index=True)
  df_dict[key_set[0]] = df0
  key_set.remove(key_set[1])
df = df_dict[14] # our one big dataframe after merging

There are two University of Miami's in our datasets. One in Florida and another in Ohio. We wanted to strip the parentheses off the location to help for a later step which also deals with parsing info out of parentheses.

In [ ]:
df['Team'] = df['Team'].str.replace('Miami \(FL\)', 'Miami FL', regex=True)
df['Team'] = df['Team'].str.replace('Miami \(OH\)', 'Miami OH', regex=True)

The datasets mistakenly mixed up the data for two features. Data for the number of 4th down attempts a team had was placed under the feature for the number of 4th down conversions that team had and vice versa. So we need to swap the data to the correct columns.

In [ ]:
temp_data = df['X4th.Attempts'].copy()
df['X4th.Attempts'] = df['X4th.Conversions']
df['X4th.Conversions'] = temp_data

## Initial Data Exporation

We wanted to examine if the notion on college football becoming more offensive is actually true. Offensive yards per game and total touchdowns are good measures of offensive productivity, so we can graph the yearly averages of these features.

In [ ]:
data = df.groupby('year')['Off.Yards.per.Game'].mean()
data.plot(kind='bar')
plt.xlabel('year')
plt.ylabel('average offensive yards per game')
plt.show()
data = df.groupby('year')['Points.Per.Game'].mean()
data.plot(kind='bar')
plt.xlabel('year')
plt.ylabel('average points per game')
plt.show()

We were also curious in validating on if passing is becmoing more prominent and the value of rushing is fading. Pass yards per game and rush yards per game can give us a good idea on this topic.

In [ ]:
data = df.groupby('year')[['Pass.Yards.Per.Game', 'Rushing.Yards.per.Game']].mean()
data.plot(kind='bar', stacked=True)
plt.xlabel('year?')
plt.ylabel('distribution of total yards')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

Staying with the theme on if teams are becoming more aggressive offensively, we can analyze the average amount of 4th down attempts teams are taking each year. The dramatic drop in 2020 is due to Covid-19 impacting the number of games many teams played that year.

In [ ]:
data = df.groupby('year')['X4th.Attempts'].mean()
data.plot(kind='line')
plt.xlabel('year?')
plt.ylabel('average 4th down attempts')
plt.show()

Concern for players' health is a growing trend in sports nowdays. Defense is where players exhaust alot of energy, so we were interested if there is any trend in the number of defensive snaps teams are taking each year. The dramatic drop in 2020 is due to Covid-19 impacting the number of games many teams played that year.

In [ ]:
data = df.groupby('year')['Def.Plays'].mean()
data.plot(kind='line')
plt.xlabel('year?')
plt.ylabel('average defensive plays')
plt.show()

There has also been a movement to protect the quarterback more, which has led to tackles on the quarterback sometimes being ruled a penalty instead of a sack for the defense. We can graph the yearly averages of penalty yards per game and sacks per game to see if its true that penalties are increasing due to new rules, resulting in less sacks for the defense, or if the extent of that is overblown.

In [ ]:
data = df.groupby('year')['Penalty.Yards.Per.Game'].mean()
data.plot(kind='line')
plt.xlabel('year')
plt.ylabel('average penalty yards per game')
plt.show()
data = df.groupby('year')['Average.Sacks.per.Game'].mean()
data.plot(kind='line')
plt.xlabel('year')
plt.ylabel('average sacks per game')
plt.show()

Also staying on the topic of health, there have been rules put in place to deincentivize teams from trying to return kickoffs as they can be one of the most dangerous plays in football due to players running full speed at each other from oppositie sides of the field. We wanted to examine if there is an observable decrease following these new rules.

In [ ]:
data = df.groupby('year')['Kickoff.Return.Yards'].mean()
data.plot(kind='line')
plt.xlabel('year')
plt.ylabel('average kick return yeards')
plt.show()
data = df.groupby('year')['Kickoffs.Returned'].mean()
data.plot(kind='line')
plt.xlabel('year')
plt.ylabel('average kickoffs returned')

## Feature Engineering

There are features that record the number of pass attempts and and pass completions a team had. We can make a completion percentage feature out of this to express the odds a pass will be succesfully completed when a team chooses to throw. 

We will drop the number of completions since we now have this completion percentage feature, but we will keep the number of pass attempts a team has since that describes their play style, which could determine how successful they are.

In [ ]:
df['complt perct'] = df['Pass.Completions'] / df['Pass.Attempts']
df = df.drop(columns='Pass.Completions')

There are features that record the number of pass attempts faced and and pass completions allowed a team had. We can make an opponent completion percentage feature out of this to express the odds a pass will be succesfully completed against this team when they face a pass.

We will drop the number of completions allowed since we now have this opponent completion percentage feature, but we will keep the number of pass attempts a team faces since that describes how opponents play against this team, which could determine how successful they are.

In [ ]:
df['opp complt perct'] = df['Opp.Completions.Allowed'] / df['Opp.Pass.Attempts']
df = df.drop(columns='Opp.Completions.Allowed')

The best teams when they are in the redzone(the last 20 yards before the endzone) score touchdowns(6 points) when they are that close opposed to settling for field goals(3 points) or not scoring at all. We can make a feature that expresses how often a team scores a touchdown when they are in the redzone. 

We will keep the amount of redzone attempts feature beacuse that tells us how many times a team gets into a close scoring poistion, which can say alot about how good or bad a team is.

In [ ]:
df['redzone td perct'] = (df['Redzone.Rush.TD'] + df['Redzone.Pass.TD']) / df['Redzone.Attempts']

It is important to also know on what percentage of trips to the redzone a team settles for a field goal, so we can create a feature for that.

We can then drop the feature for the number of redzone field goals a team made after creating our redzone filed goal percentage feature.

In [ ]:
df['redzone fg perct'] = df['Redzone.Field.Goals.Made'] / df['Redzone.Attempts']
df = df.drop(columns='Redzone.Field.Goals.Made')

We can then lastly make a feature to express the percentage of trips to the redzone a teams fails to score using the features we created above.

In [ ]:
df['redzone empty perct'] = 1 - df['redzone td perct'] - df['redzone fg perct']

We can apply all the same logic above to create features for the opponents redzone statistis which can give us an idea of how good this teams' redzone defense is and if they can force their opponent into settling for field goals instead of scoring touchdowns.

In [ ]:
df['opp redzone td perct'] = (df['Opp.Redzone.Rush.TD.Allowed'] + df['Opp.Redzone.Pass.Touchdowns.Allowed']) / df['Opp.Redzone.Attempts']
df['opp redzone fg perct'] = df['Opp.Redzone.Field.Goals.Made'] / df['Opp.Redzone.Attempts']
df = df.drop(columns='Opp.Redzone.Field.Goals.Made')
df['opp redzone empty perct'] = 1 - df['opp redzone td perct'] - df['opp redzone fg perct']

Big plays are very critical to have as an offense and limit as a defense in football. We can create features expressing for a team the number of touchdowns they scored and allowed outside the redzone(scores of over 20 yards).

We can subsequently drop the amount of redzone rush and pass touchdowns a team has had and allowed after creating the redzone touchdown percentage feature above and this non redzone touhchdown feature below.

In [ ]:
df['non redzone td'] = df['Off.TDs'] - (df['Redzone.Rush.TD'] + df['Redzone.Pass.TD'])
df['non redzone td allowed'] = df['Off.TDs.Allowed'] - (df['Opp.Redzone.Rush.TD.Allowed'] + df['Opp.Redzone.Pass.Touchdowns.Allowed'])
columns_to_drop = ['Redzone.Rush.TD', 'Redzone.Pass.TD', 'Opp.Redzone.Rush.TD.Allowed', 'Opp.Redzone.Pass.Touchdowns.Allowed']
df = df.drop(columns=columns_to_drop)

The number of games teams play can vary based on if they qualified for their conference championship or not, and during the Covid-19 pandemic, some conferences played a drastically different amount of games than others. For these reasons, it would make more sense to use win percentage rather than just number of wins in our model.

We can subsequently drop the number of wins, lossess, and games played from our dataframe after creating our win percentage feature.

In [ ]:
df['win perct'] = df['Win'] / df['Games']
columns_to_drop = ['Win', 'Loss', 'Games']
df = df.drop(columns=columns_to_drop)

The conference a team is plays a big role in how the commitee ranks them. Most of the games teams play are against teams in the same conference, and some conferences have historically had higher performing teams. As a result, teams in these more elite conferences often have harder schedules, and thus it is critical to take into consideration the conference a team plays in when ranking them.

In our datasets, the conference is not its own feature, it is attached to the feature for the team's name. Hence, we parse the conference from the team's name and add a column for the conference the team is in.

After we parse the conference from the team name feature and create the conference feature, we can remove the conference from the team name.

In [ ]:
df['conference'] = df['Team'].apply(lambda x: x[x.find('(') + 1: x.find(')')] if '(' in x and ')' in x else x)
df['Team'] = df['Team'].apply(lambda x: x[:x.find('(')] if '(' in x else x)
df['Team'] = df['Team'].str[:-1]

There were a couple individual records with missing/wrong conferences, so we manually set the teams' conference to its correct one.

In [ ]:
df.loc[df['conference'] == 'Independent', 'conference'] = 'FBS Independent'
df.loc[(df['conference'] == '') & (df['Team'] == 'Ole Miss'), 'conference'] = 'SEC'
df.loc[(df['conference'] == '') & (df['Team'] == 'Pittsburgh'), 'conference'] = 'ACC'

Our class label is if the team was ranked in the top 25 or not for the given year, so we utilize the lists below to add a column which holds a 1 if the team was ranked in the top 25 that year and a 0 if not.

After we create the feature that says if a team was in the top 25 or not for the given year, we can drop the year column.

In [ ]:
# lists of the CFP Commitee's top 25 ranking each year

top25_14 = ['Alabama', 'Oregon', 'Florida St.', 'Ohio St.', 'Baylor',
            'TCU', 'Mississippi St.', 'Michigan St.', 'Ole Miss', 'Arizona',
            'Kansas St.', 'Georgia Tech', 'Georgia', 'UCLA', 'Arizona St.',
            'Missouri', 'Clemson', 'Wisconsin', 'Auburn', 'Boise St.',
            'Louisville', 'Utah', 'LSU', 'Southern California', 'Minnesota'] # top 25 from 2014

top25_15 = ['Clemson', 'Alabama', 'Michigan St.', 'Oklahoma', 'Iowa',
            'Stanford', 'Ohio St.', 'Notre Dame', 'Florida St.', 'North Carolina',
            'TCU', 'Ole Miss', 'Northwestern', 'Michigan', 'Oregon',
            'Oklahoma St.', 'Baylor', 'Houston', 'Florida', 'LSU',
            'Navy', 'Utah', 'Tennessee', 'Temple', 'Southern California'] # top 25 from 2015

top25_16 = ['Alabama', 'Clemson', 'Ohio St.', 'Washington', 'Penn St.',
            'Michigan', 'Oklahoma', 'Wisconsin', 'Southern California', 'Colorado',
            'Florida St.', 'Oklahoma St.', 'Louisville', 'Auburn', 'Western Mich.',
            'West Virginia', 'Florida', 'Stanford', 'Utah', 'LSU',
            'Tennessee', 'Virginia Tech', 'Pittsburgh', 'Temple', 'Navy'] # top 25 from 2016

top25_17 = ['Clemson', 'Oklahoma', 'Georgia', 'Alabama', 'Ohio St.',
            'Wisconsin', 'Auburn', 'Southern California', 'Penn St.', 'Miami FL',
            'Washington', 'UCF', 'Stanford', 'Notre Dame', 'TCU',
            'Michigan St.', 'LSU', 'Washington St.', 'Oklahoma St.', 'Memphis',
            'Northwestern', 'Virginia Tech', 'Mississippi St.', 'NC State', 'Boise St.'] # top 25 from 2017

top25_18 = ['Alabama', 'Clemson', 'Notre Dame', 'Oklahoma', 'Georgia',
            'Ohio St.', 'Michigan', 'UCF', 'Washington', 'Florida',
            'LSU', 'Penn St.', 'Washington St.', 'Kentucky', 'Texas',
            'West Virginia', 'Utah', 'Mississippi St.', 'Texas A&M', 'Syracuse',
            'Fresno St.', 'Northwestern', 'Missouri', 'Iowa St.', 'Boise St.'] # top 25 from 2018

top25_19 = ['LSU', 'Ohio St.', 'Clemson', 'Oklahoma', 'Georgia',
            'Oregon', 'Baylor', 'Wisconsin', 'Florida', 'Penn St.',
            'Utah', 'Auburn', 'Alabama', 'Michigan', 'Notre Dame',
            'Iowa', 'Memphis', 'Minnesota', 'Boise St.', 'Appalachian St.',
            'Cincinnati', 'Southern California', 'Navy', 'Virginia', 'Oklahoma St.'] # top 25 from 2019

top25_20 = ['Alabama', 'Clemson', 'Ohio St.', 'Notre Dame', 'Texas A&M',
            'Oklahoma', 'Florida', 'Cincinnati', 'Georgia', 'Iowa St.',
            'Indiana', 'Coastal Carolina', 'North Carolina', 'Northwestern', 'Iowa',
            'BYU', 'Southern California', 'Miami FL', 'Louisiana', 'Texas',
            'Oklahoma St.', 'San Jose St.', 'NC State', 'Tulsa', 'Oregon'] # top 25 from 2020

top25_dict = {14: top25_14, 15: top25_15, 16: top25_16, 17: top25_17, 18: top25_18,
              19: top25_19, 20: top25_20} # dictionary to be able to grab rankings for a desired year

df['top 25'] = df.apply(lambda row: 1 if row['Team'] in top25_dict.get(row['year'], []) else 0, axis=1)
df = df.drop(columns=['year'])

The below features are being dropped beacause there is already a feature in our dataframe that is a calculation of these features.

In [ ]:
columns_to_drop = ['Off.Plays', 'Def.Plays', 'X4th.Conversions', 'Opp.4th.Conversion',
                   'Kickoffs.Returned', 'Kickoff.Return.Yards', 'Punt.Returns', 'Net.Punt.Return.Yards',
                   'Opp.Punt.Returns', 'Opp.Net.Punt.Return.Yards', 'Redzone.Points', 'Redzone.Points.Allowed',
                   'X3rd.Attempts', 'X3rd.Conversions', 'Opp.3rd.Attempt', 'Opp.3rd.Conversion']
df = df.drop(columns=columns_to_drop)

## Additional Data Exploration

To show why we would want to have a feature that says what conference a team is in, the below plot displays the distribution of ranked teams based on what conference they are in. As we can see, teams in the top 25 primarily come from a couple conferences.

In [ ]:
data = df.groupby('conference')['top 25'].sum()
data = data.sort_values(ascending=False)
data.plot(kind='bar')
plt.xlabel('conference')
plt.ylabel('number of teams in top 25 from 2014-2020')
plt.show()

This analysis of conferences can be taken further by looking at the distribution of the top 25 based on conference to see on average what proportion of ranked teams come from each conference.

In [ ]:
data = df.groupby('conference')['top 25'].sum()
data = data.sort_values(ascending=False)
data = data / num_years
data.plot(kind='pie')
plt.title('proportion of the top 25 that is made up by each conference on average')
plt.show()

A few important features we engineered were the percenatges of how teams scored points when they were in the redzone, so we can plot the average percentage of time ranked versus unranked teams score touchdowns, field goals, or come up empty when they are in the redzone.

In [ ]:
data = df.groupby('top 25')[['redzone td perct', 'redzone fg perct', 'redzone empty perct']].mean()
ax = data.plot(kind='bar', stacked=True)
plt.xlabel('top 25?')
plt.ylabel('avg redzone percentages')
plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.show()

One of the most talked-about statistics in the media is the turnover margin. The turnover margin is the difference between the number of times a team took the ball away from their opponent and the number of times they gave the ball to their opponent. Good teams are known to have positive turnover margins, and we wanted to confirm that by plotting the average turnover margin for ranked teams versus those teams not in the top 25.

In [ ]:
data = df.groupby('top 25')['Turnover.Margin'].mean()
data.plot(kind='bar')
plt.xlabel('top 25?')
plt.ylabel('avg turnover margin')
plt.show()

## Modeling

In [ ]:
one_hot = pd.get_dummies(df['conference'])
one_hot = one_hot.astype('int')
#df = pd.get_dummies(df, columns = ['conference'])
df = df.drop('conference', axis = 1)
df.join(one_hot)

In [ ]:
no_string_df = df.drop(['Team'], axis=1)

def convert_time(str_time):
    minutes, seconds = str_time.split(':')
    return (int(minutes) * 60) + int(seconds)

no_string_df['Time.of.Possession'] = no_string_df['Time.of.Possession'].apply(lambda x: convert_time(x))
no_string_df['Average.Time.of.Possession.per.Game'] = no_string_df['Average.Time.of.Possession.per.Game'].apply(lambda x: convert_time(x))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()
pca = PCA()
scaled_df = scaler.fit_transform(no_string_df)
scaled_df = pd.DataFrame(scaled_df, columns=no_string_df.columns)
scaled_df

## Results

In [ ]:
features = no_string_df.drop('top 25', axis=1)
labels = no_string_df['top 25'].values.ravel()
scaled_features = scaled_df.drop('top 25', axis=1)
scaled_labels = scaled_df['top 25'].values.ravel()

Decision Trees with different parameters

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

clf = DecisionTreeClassifier()
param_grid = {'max_depth': [5, 10, 15, 20], 'min_samples_leaf': [5, 10, 15, 20], 'max_features': [5, 10, 15]}
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
nested_scores = cross_val_score(grid_search, features, labels, cv=5)
print("Average accuracy:", nested_scores.mean() * 100, "%")

Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
scores = cross_val_score(gnb, features, labels, cv=10, scoring='accuracy')
print("Average accuracy:", scores.mean() * 100, "%")

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold

param_grid = {'pca__n_components': list(range(5, 19)), 'knn__n_neighbors': list(range(1, 25))}

gr_sch = GridSearchCV(pl, param_grid, cv = 5)
gr_sch.fit(features, labels)
outer_loop = KFold(n_splits = 5, shuffle = True, random_state = 21)
nested_scores = cross_val_score(gr_sch, features, labels, cv = outer_loop)
accuracy = nested_scores.mean()
print("Nested CV accuracy:", accuracy * 100, "%")

SVM

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.svm import SVC

svm_pl = Pipeline([('scaler', StandardScaler()), ('svc', SVC())])
prm_grd = { 'svc__C': [0.1, 1, 10, 100], 'svc__kernel': ['linear', 'rbf', 'poly']}
gr_sch = GridSearchCV(svm_pl, prm_grd, cv = 5)
predictions = cross_val_predict(gr_sch, features, labels, cv = outer_loop)
accuracy = cross_val_score(gr_sch, features, labels, cv = 5, scoring = 'accuracy').mean()
print("Accuracy:", accuracy * 100, "%")

Neural Nets

In [ ]:
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier

@ignore_warnings(category=ConvergenceWarning)
def run_nn():
    pl = Pipeline([('scaler', StandardScaler()), ('mlp', MLPClassifier(max_iter = 1000, random_state = 21))])
    prm_grd = {'mlp__hidden_layer_sizes': [(30,), (40,), (50,), (60,)], 'mlp__activation': ['logistic', 'tanh', 'relu']}
    grid_search = GridSearchCV(pl, prm_grd, cv=5)
    accuracy = cross_val_score(grid_search, features, labels, cv = 5, scoring = 'accuracy').mean()
    print("Accuracy:", accuracy * 100, "%")
  

run_nn()

Random Forest Ensemble

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

gnb = GaussianNB()
predicted_labels = cross_val_predict(gnb, features, labels, cv=10)
rf_clf = RandomForestClassifier(random_state = 21)
prm_grd = {'n_estimators': [50, 100, 150]}
grd_sch = GridSearchCV(rf_clf, prm_grd, cv = 5)
predictions = cross_val_predict(grd_sch, features, labels, cv = 5)
cls_rpt = classification_report(labels, predicted_labels)
print(cls_rpt)